# Project

## Introduction to the Project
The S&P 500 (Standard & Poor's 500) is a stock market index that tracks the performance of 500 of the largest publicly traded companies in the United States. It is widely regarded as one of the best representations of the U.S. stock market and economy. Over the long term, the S&P 500 has shown consistent growth, making it a key focus for long-term investors. However, it can also experience significant volatility in the short term.

In this project, we will make our first attempt to build a momentum-based strategy to trade the S&P 500 index. At the end of the project, you will have built a program that you can later expand and customise to suit your needs. We will use the python packages numpy, scipy and sqlite3, among others, in this project.

Tips: Review the code snippets that we went through during the course. Reorganize them and you are half-way done! Try experimenting with different configurations of the confidence interval, the lookback window and the forecast horizon. Be brave and experiment with different ways of deciding the position size. You may be surprised by your talent!

Re-organize your code from the exercises into a Python program that 
1. read prices from a database
2. calibrate a GBM model on each day when new prices are received.
3. forecast the price in e.g. 10 days and
   1. calculate the confidence interval of the forecast
   2. calculate the expected shortfall from the price forecast
4. code your trading signals using the price forecast and the expected shortfall.
5. store your postions into the *positions* table after each trade.
6. produce a 1-year backtest record from 2020-06-01 to 2021-05-31.

**Hint**

1. Collect all the code related to the GBM model into a class

In [2]:
import csv
import datetime
import sqlite3
from contextlib import closing

import numpy as np
import plotly.express as px
from scipy.stats import norm

np.random.seed(42)

In [3]:
class GBM:
    def __init__(self):
        self.mu: float = np.nan
        self.sigma: float = np.nan
        self.rng = np.random.default_rng()

    def simulate(self, n: int, k: int, dt: float, s0: float) -> np.ndarray:
        sqrt_dt = 1 / np.sqrt(n)
        traj = np.full((n + 1, k), np.nan)
        drift = (self.mu - self.sigma**2 / 2) * np.linspace(1, n, n) * dt
        for i in range(k):
            W = sqrt_dt * np.cumsum(norm.rvs(size=n))
            traj[1:, i] = s0 * np.exp(drift + self.sigma * W)
            traj[0, i] = s0
        return traj

    def calibrate(self, trajectory: np.ndarray, dt: float) -> None:
        increments = np.diff(np.log(trajectory))
        moments = [0, 0]
        n_iter = 10
        for _ in range(n_iter):
            X = self.rng.choice(increments, size=len(increments) // 2)
            moments[0] += np.mean(X) / n_iter
            moments[1] += np.mean(X**2) / n_iter
        std = np.sqrt(moments[1] - moments[0] ** 2)
        self.sigma = std / np.sqrt(dt)
        self.mu = moments[0] / dt + self.sigma**2 / 2

    def forecast(
        self, latest: float, t: float, confidence: float
    ) -> dict[str, float | np.ndarray]:
        mu = (self.mu - self.sigma**2 / 2) * t
        s: float = self.sigma * np.sqrt(t)
        q = norm.ppf([1 - confidence / 2, 1 + confidence / 2], loc=mu, scale=s)
        return {
            "confidence": confidence,
            "expected": latest * np.exp(self.mu * t),
            "interval": latest * np.exp(q),
        }

    def expected_shortfall(self, t: float, confidence: float) -> float:
        m = (self.mu - self.sigma**2 / 2) * t
        s: float = self.sigma * np.sqrt(t)
        es = -m + s * norm.pdf(norm.ppf(confidence)) / (1 - confidence)
        return es

In [4]:
# Test GBM model
n = 500
k = 1
dt = 1 / 250
s0 = 100

model = GBM()
model.mu = 0.3
model.sigma = 0.2
simulated = model.simulate(n=n, k=k, dt=dt, s0=s0)
simulated = simulated[:, 0]

model2 = GBM()
model2.calibrate(trajectory=simulated, dt=dt)

print(f"Calibrated: mu = {model2.mu}, sigma = {model2.sigma}")

Calibrated: mu = 0.3206844394026786, sigma = 0.13964822640218238


2. Write a function that prepares the database for trading, i.e.
   1. load the historical prices into the *prices* table
   2. create the *positions* table
   3. initialize the *positions* table with the your initial cash reserve. The initial *time_of_trade* can be any date before the earliest possible trading date.

    Call this function *prepare*.

In [5]:
def prepare(cs: sqlite3.Cursor, conn: sqlite3.Connection):
    cs.execute(
        """
    create table if not exists prices (
    theday text primary key,
    price real
    )
    """
    )
    cs.execute("delete from prices")
    conn.commit()
    with closing(open("05_SP500.csv")) as datafile:
        reader = csv.DictReader(datafile, fieldnames=["date", "price"], delimiter="\t")
        for row in reader:
            cs.execute(
                f"insert into prices values (\"{row['date']}\", {float(row['price'])})"
            )
    conn.commit()

    cs.execute(
        """
    create table if not exists positions (
    time_of_trade text,
    instrument text,
    quantity real,
    cash real,
    primary key (time_of_trade, instrument)
    )
    """
    )
    cs.execute(
        """
    insert or ignore into positions values
    ('1666-01-01', 'SP500', 0, 1000000)
    """
    )
    conn.commit()

In [6]:
# Test the prepare function
with closing(sqlite3.connect("SP500.db")) as conn:
    with closing(conn.cursor()) as cs:
        prepare(cs=cs, conn=conn)
        latest_prices = cs.execute("select * from prices order by theday desc limit 10")
        for item in latest_prices:
            print(item)

FileNotFoundError: [Errno 2] No such file or directory: '05_SP500.csv'

3. Write a function that determines the trade size, i.e. how many units of the instrument you would like to own when the date is *which_day* and the price forecast of the instrument is *forecast* and the expected shortfall from the same forecast is *ES*.

In [ ]:
def position_size(
    cs: sqlite3.Cursor,
    which_day: datetime.date,
    forecast_interval: np.ndarray,
    es: float,
    settings: dict[str, float],
) -> int:
    cs.execute(
        f"""
    select quantity, cash from positions
    where instrument = 'SP500'
    and time_of_trade < '{which_day}'
    order by time_of_trade desc
    limit 1
    """
    )
    qty, cash = cs.fetchall()[0]
    cs.execute(
        f"""
    select price from prices
    where theday <= '{which_day}'
    order by theday desc
    limit 1
    """
    )
    price = cs.fetchall()[0][0]
    capital = cash + qty * price

    # Prevent division by very small es values which would lead to extreme risk-taking
    if abs(es) < 0.001:
        es = 0.001 if es >= 0 else -0.001
    exposure = capital * settings["risk_sizing"] / es

    if price < forecast_interval[0]:
        # Ensure we don't buy more than we can afford - no bankruptcy allowed
        desired_shares = round(exposure / price)
        max_shares = int(cash / price)
        return min(desired_shares, max_shares)
    elif price > forecast_interval[1]:
        # Ensure we don't sell more than we have - no short selling beyond current holdings
        desired_short = -round(exposure / price)
        max_short = -int(qty)
        return max(desired_short, max_short)
    else:
        return 0

4. Write a function that, for a given date, calibrates a GBM model to the data prior to that date and that forecasts the price in 10 days. Call this function *analyse*.

In [ ]:
def analyse(
    cs: sqlite3.Cursor, which_day: datetime.date, settings: dict[str, float]
) -> int:
    cs.execute(
        f"""
    select price from prices where theday <= '{which_day}'
    order by theday desc limit {settings['lookback']}
    """
    )
    p = np.flipud(np.asarray(cs.fetchall())).flatten()
    model = GBM()
    dt = 1.0 / 252
    model.calibrate(trajectory=p, dt=dt)
    n = settings["forecast_days"]
    t = n * dt
    forecast = model.forecast(latest=p[-1], t=t, confidence=settings["confidence"])
    es = model.expected_shortfall(t=t, confidence=settings["confidence"])
    return position_size(
        cs=cs,
        which_day=which_day,
        forecast_interval=forecast["interval"],
        es=es,
        settings=settings,
    )

In [ ]:
# Test the analyse function
settings = {
    "confidence": 0.4,
    "min_trade_size": 10,
    "risk_sizing": 0.04,
    "lookback": 120,
    "forecast_days": 10,
}
with closing(sqlite3.connect("SP500.db")) as conn:
    with closing(conn.cursor()) as cs:
        test_dates = [datetime.date(2021, 5, 9), datetime.date(2021, 5, 14)]
        positions = [np.nan, np.nan]
        for i in range(2):
            positions[i] = analyse(cs=cs, which_day=test_dates[i], settings=settings)
            print(f"{positions[i]} shares advised on {test_dates[i]}.")

-141 shares advised on 2021-05-09.
356 shares advised on 2021-05-14.


5. The main loop of the program: Loop over the dates in the backtest period and use the *analyse* function to decide what to do on each day. Call this function *main*.

In [ ]:
def main(
    cs: sqlite3.Cursor,
    conn: sqlite3.Connection,
    begin_on: datetime.date,
    settings: dict[str, float],
) -> None:
    cs.execute(f"select theday from prices where theday >= '{begin_on}'")
    days = [d[0] for d in cs.fetchall()]
    asset = {"old": np.nan, "new": np.nan}
    cash = {"old": np.nan, "new": np.nan}
    cs.execute("delete from positions where time_of_trade > '2020-01-01'")
    for d in days:
        asset["new"] = analyse(cs=cs, which_day=d, settings=settings)
        cs.execute(
            f"""
        select quantity, cash from positions
        where time_of_trade < '{d}'
        order by time_of_trade desc
        limit 1
        """
        )
        asset["old"], cash["old"] = cs.fetchall()[0]
        cs.execute(
            f"""
        select price from prices
        where theday <= '{d}'
        order by theday desc
        limit 1
        """
        )
        latest = cs.fetchall()[0][0]
        trade_size = round(asset["new"]) - round(asset["old"])
        if trade_size != 0 and abs(trade_size) > settings["min_trade_size"]:
            cash["new"] = cash["old"] - trade_size * latest
            cs.execute(
                f"""
            insert into positions values
            ('{d}', 'SP500', {round(asset['new'])}, {cash['new']})
            """
            )
        conn.commit()

6. Connect to the database and create a *cursor* object associated with the connection. Share the connection and the cursor object across the program so that you don't have to connect to and disconnect from the database in every function of the program.

In [ ]:
settings = {
    "confidence": 0.4,
    "min_trade_size": 15,
    "risk_sizing": 0.02,
    "lookback": 120,
    "forecast_days": 10,
}

if __name__ == "__main__":
    with closing(sqlite3.connect("SP500.db")) as conn:
        with closing(conn.cursor()) as cs:
            prepare(cs=cs, conn=conn)
            main(
                cs=cs, conn=conn, begin_on=datetime.date(2020, 6, 1), settings=settings
            )

In [ ]:
with closing(sqlite3.connect("SP500.db")) as conn:
    with closing(conn.cursor()) as cs:
        date_start = datetime.datetime(2020, 6, 1)
        cs.execute(
            f"""
            select theday, quantity * price + cash as wealth
            from positions as PO
            join prices as PR
            on PO.time_of_trade = (
                select time_of_trade from positions
                where time_of_trade <= PR.theday
                order by time_of_trade desc limit 1
            )
            where theday >= '{date_start}'
            order by theday
        """
        )
        records = cs.fetchall()
        records_t = [
            (
                (datetime.datetime.strptime(date_str, "%Y-%m-%d") - date_start).days,
                value,
            )
            for date_str, value in records
        ]
        w = np.asarray(records_t)

px.line(
    x=w[:, 0],
    y=w[:, 1],
    labels={"x": "Number of days of trading", "y": "Total Wealth"},
    title="Wealth Over Time",
)

### Commentary to parameters

<b>confidence</b>

Wider interval causes more conservative approach, which on several attempts produces better results. Value of just 10 % is too low, total wealth at the end was almost always 0. Funny (obvious) thing - setting it to 0.99 copies the SP500 profile and it's actually the most stable profitable long-term strategy. However, it will certainly not bring us to multiplication of initial wealth, unlike higher risk with narrower confidence interval.

<b>min_trade_size</b>

Minimum trade size is there to avoid excessive trading costs from frequent small trades (not implemented, but potentially useful). Also avoids too eager trading behavior in this simulation.

<b>risk_sizing</b>

Risk sizing means the percentage we are willing to risk from our total capital on a single trade. Lower values - more conservative trading. Initiall 5 % seemed to high to me.

<b>lookback</b>

Lower lookback number eans we consider less history for our calibration. It's worth trying to shorten it to catch more volatility in the medium/short term.

<b>forecast_days</b>

Number of days we are forecasting.